In [231]:
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df1 = pd.read_csv('datasets/players.csv')
df2 = pd.read_csv('datasets/players_teams.csv')
df3 = pd.read_csv('datasets/awards_players.csv')
df4 = pd.read_csv('datasets/teams.csv')
df5 = pd.read_csv('datasets/teams_post.csv')
df6 = pd.read_csv('datasets/coaches.csv')
df7 = pd.read_csv('datasets/series_post.csv')

def corrige_vencedor(teams, series_post):
    # Itera sobre cada rodada ('F', 'CF', 'FR') para ajustar cada fase dos playoffs
    for round_type in ['FR', 'CF', 'F']:
        # Filtra a série específica da rodada
        series_round = series_post[series_post['round'] == round_type]
        
        # Atualiza cada série individualmente
        for _, row in series_round.iterrows():
            year = row['year']
            winner_id = row['tmIDWinner']
            loser_id = row['tmIDLoser']
            
            # Define as colunas que correspondem às rodadas
            if round_type == 'FR':
                round_column = 'firstRound'
            elif round_type == 'CF':
                round_column = 'semis'
            elif round_type == 'F':
                round_column = 'finals'
            
            # Marca o time vencedor como "W" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == winner_id), round_column] = 'W'
            
            # Marca o time perdedor como "L" na rodada correspondente
            teams.loc[(teams['year'] == year) & (teams['tmID'] == loser_id), round_column] = 'L'
    
    return teams

teams_file = corrige_vencedor(df4, df7)

players_teams_file = df2.drop(columns=['lgID'])
players_file = df1[df1['pos'].notna() & (df1['pos'] != '')]
players_file = players_file.drop(columns=['firstseason', 'lastseason', 'deathDate', 'collegeOther'])
players_file['college'] = players_file['college'].apply(lambda x: 1 if pd.notnull(x) else 0)
merged_df = pd.merge(players_teams_file, players_file, left_on='playerID', right_on='bioID', how='left')
merged_df = merged_df.drop(columns=['bioID'])
awards_players_file = df3.drop(columns=['lgID'])
teams_file = df4.drop(columns=['lgID', 'divID', 'tmORB','tmDRB','tmTRB','opptmORB','opptmDRB','opptmTRB','seeded'])
teams_file['playoff'] = teams_file['playoff'].apply(lambda x: 1 if x=='Y' else 0)

team_post_file = df5.drop(columns=['lgID'])
series_post_file = df7.drop(columns=['lgIDWinner', 'lgIDLoser'])
coaches_file = df6.drop(columns=['lgID'])


awards_grouped = awards_players_file.groupby(['playerID', 'year'])['award'].apply(list).reset_index()
awards_grouped['award'] = awards_grouped['award'].apply(lambda x: x if isinstance(x, list) else [])

merged_df = pd.merge(merged_df, awards_grouped, on=['playerID', 'year'], how='left')
merged_df['award'] = merged_df['award'].apply(lambda x: x if isinstance(x, list) else [])
merged_df = pd.merge(merged_df, teams_file, on=['tmID','year'], how = 'left')

merged_df = merged_df.drop(columns=['franchID', 'name'])

merged_df = pd.merge(merged_df, team_post_file, on=['tmID','year'], how = 'left')

In [232]:
player_count_per_team_year = merged_df.groupby(['year', 'tmID'])['playerID'].nunique().reset_index()
player_count_per_team_year.columns = ['Year', 'Team', 'PlayerCount']

# print(player_count_per_team_year)

# Calculate min, max, and average player count for each year
summary_stats = player_count_per_team_year.groupby('Year')['PlayerCount'].agg(['min', 'max', 'mean']).reset_index()
summary_stats.columns = ['Year', 'MinPlayerCount', 'MaxPlayerCount', 'AvgPlayerCount']

In [233]:
awards_coaches_file = df3.rename(columns={'playerID': 'coachID'})
coach_awards = awards_coaches_file[awards_coaches_file['award'] == 'Coach of the Year']
coach_awards_grouped = coach_awards.groupby(['coachID', 'year'])['award'].apply(list).reset_index()
coaches_file = pd.merge(coaches_file, coach_awards_grouped, on=['coachID', 'year'], how='left')

In [234]:
avg_oRebounds_by_pos = merged_df.groupby('pos')['oRebounds'].mean().reset_index()
avg_dRebounds_by_pos = merged_df.groupby('pos')['dRebounds'].mean().reset_index()

In [235]:
merged_df = merged_df.drop(columns=['rebounds', 'PostRebounds'])
merged_df = merged_df.rename(columns={'GP_x': 'GP_player', 'GP_y': 'GP_team'})

In [236]:
grouped = merged_df.groupby('year').agg({
    'o_pts': 'sum',
    'o_fga': 'sum',
    'o_oreb': 'sum',
    'o_to': 'sum',
    'o_fta': 'sum',
    'o_asts': 'sum',
    'o_fgm' : 'sum',
    'o_ftm': 'sum',
    'o_dreb':'sum',
}).reset_index()

grouped['VOP'] = grouped['o_pts'] / (grouped['o_fga'] - grouped['o_oreb'] + grouped['o_to'] + 0.44 * grouped['o_fta'])
grouped['factor'] = (2 / 3) - (0.5 * (grouped['o_asts'] / grouped['o_fgm'])) / (2 * (grouped['o_fgm'] / grouped['o_ftm']))
grouped['DRB%'] = (grouped['o_dreb'] - grouped['o_oreb']) / grouped['o_dreb']

uPER_df = merged_df.groupby(['playerID', 'year']).agg({
    'minutes': 'sum',     
    'threeMade': 'sum',   
    'assists': 'sum',     
    'fgMade': 'sum',      
    'ftMade': 'sum',      
    'turnovers': 'sum',   
    'fgAttempted': 'sum', 
    'ftAttempted': 'sum', 
    'dRebounds': 'sum',   
    'oRebounds': 'sum',   
    'steals': 'sum',      
    'blocks': 'sum',      
    'PF': 'sum'           
}).reset_index()

uPER_df = uPER_df.merge(grouped[['year', 'VOP', 'factor', 'DRB%']], on='year')

uPER_df['TRB'] = uPER_df['dRebounds'] + uPER_df['oRebounds']

uPER_df['uPER'] = (1 / uPER_df['minutes']) * (
    uPER_df['threeMade'] +
    (2/3) * uPER_df['assists'] +
    (2 - uPER_df['factor'] * (uPER_df['assists'] / uPER_df['fgMade'])) * uPER_df['fgMade'] +
    (uPER_df['ftMade'] * 0.5 * (1 + (1 - (uPER_df['assists'] / uPER_df['fgMade'])) + (2/3) * (uPER_df['assists'] / uPER_df['fgMade']))) -
    uPER_df['VOP'] * uPER_df['turnovers'] -
    uPER_df['VOP'] * uPER_df['DRB%'] * (uPER_df['fgAttempted'] - uPER_df['fgMade']) -
    uPER_df['VOP'] * 0.44 * (0.44 + (0.56 * uPER_df['DRB%'])) * (uPER_df['ftAttempted'] - uPER_df['ftMade']) +
    uPER_df['VOP'] * (1 - uPER_df['DRB%']) * uPER_df['TRB'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['oRebounds'] +
    uPER_df['VOP'] * uPER_df['steals'] +
    uPER_df['VOP'] * uPER_df['DRB%'] * uPER_df['blocks'] -
    uPER_df['PF'] * ((grouped['o_ftm'].mean() / grouped['o_pts'].mean()) - 0.44 * (grouped['o_fta'].mean() / grouped['o_pts'].mean()) * uPER_df['VOP'])
)

lg_uPER = uPER_df.groupby('year')['uPER'].mean().reset_index()
lg_uPER.rename(columns={'uPER': 'lg_uPER'}, inplace=True)

uPER_df = uPER_df.merge(lg_uPER, on='year')

uPER_df['PER'] = uPER_df['uPER'] * (15 / uPER_df['lg_uPER'])


In [237]:
per_to_merge = uPER_df[['playerID', 'year', 'PER']]
merged_df = merged_df.merge(per_to_merge, on=['playerID', 'year'], how='left')

merged_df['TS%'] = (merged_df['points'] / (2 * (merged_df['fgAttempted'] + 0.44 * merged_df['ftAttempted'])))*100
merged_df['eFG%'] = ((merged_df['fgMade'] + 0.5 * merged_df['threeMade']) / merged_df['fgAttempted'])*100
merged_df['stocks'] = (merged_df['steals'] + merged_df['blocks'])

merged_df['PER'] = merged_df['PER'].fillna(0)
merged_df['TS%'] = merged_df['TS%'].fillna(0)
merged_df['eFG%'] = merged_df['eFG%'].fillna(0)
merged_df['stocks'] = merged_df['stocks'].fillna(0)

#Equipas que não foram aos playoffs
merged_df['W'] = merged_df['W'].fillna(0)
merged_df['L'] = merged_df['L'].fillna(0)

In [238]:
def box_plot_for_each_column(dataset):
    numeric_columns = dataset.select_dtypes(include='number')
    if numeric_columns.empty:
        print("No numeric columns found in the dataset.")
    else:
        numeric_columns.boxplot(figsize=(10, 6))
        plt.title("Boxplot for all numeric columns")
        plt.xticks(rotation=45)  # Rotation in x, if necessary
        plt.show()

def pearson_correlation(dataset, size_x, size_y):
    numeric_columns = dataset.select_dtypes(include='number')
    
    if numeric_columns.empty:
        print("Nenhuma coluna numérica encontrada no dataset.")
    else:
        # Correlation matrix
        correlation_matrix = numeric_columns.corr()

        # View
        plt.figure(figsize=(size_x, size_y))
        sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
        plt.title('Pearson-correlation')
        plt.show()

def bar_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            value_counts = non_numeric_columns[column].value_counts()
            plt.figure(figsize=(10, 6))
            value_counts.plot(kind='bar')
            plt.title(f"Bar chart for '{column}'")
            plt.xlabel(column)
            plt.ylabel("Count")
            plt.xticks(rotation=45)
            plt.tight_layout()  # Adjust layout to prevent overlap
            plt.show()

# Pie-chart for each column
def pie_chart_for_each_column(dataset):
    non_numeric_columns = dataset.select_dtypes(exclude='number')
    
    if non_numeric_columns.empty:
        print("Any non-numeric columns found in the dataset.")
    else:
        for column in non_numeric_columns.columns:
            # Count elements from different categories
            category_counts = dataset[column].value_counts()
            
            # Pie-chart
            plt.figure(figsize=(6, 6))
            category_counts.plot.pie(autopct='%1.1f%%', startangle=140)
            plt.title(f'Distribution of {column}')
            plt.ylabel('')  # Remove o rótulo do eixo Y
            plt.show()


#box_plot_for_each_column(merged_df)
#box_plot_for_each_column(coaches_file)
#box_plot_for_each_column(series_post_file)



In [239]:
#pearson_correlation(merged_df, 100, 80)
#pearson_correlation(coaches_file, 8, 6)
#pearson_correlation(series_post_file, 8, 6)

In [240]:
#bar_chart_for_each_column(merged_df)
#bar_chart_for_each_column(coaches_file)
#bar_chart_for_each_column(series_post_file)

In [241]:
#pie_chart_for_each_column(merged_df)
#pie_chart_for_each_column(coaches_file)
#pie_chart_for_each_column(series_post_file)

In [242]:
def replaceGameResults(column):
    return column.apply(lambda value: '100' if value == 'W' else '010' if value == 'L' else '001')

# Aplicar a função para cada coluna específica
merged_df['firstRound'] = replaceGameResults(merged_df['firstRound'])
merged_df['semis'] = replaceGameResults(merged_df['semis'])
merged_df['finals'] = replaceGameResults(merged_df['finals'])


In [243]:
if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df.to_csv('cleanDatasets/players_and_teams.csv', index=False)
coaches_file.to_csv('cleanDatasets/coaches_and_awards.csv', index=False)
series_post_file.to_csv('cleanDatasets/series_post.csv', index=False)

In [244]:
merged_df2 = merged_df.drop(columns=['minutes','points','threeMade','assists','fgMade','turnovers','fgAttempted','ftAttempted','oRebounds','steals','blocks','PF','o_ftm','o_pts','o_fta','o_pts','o_fga','o_oreb','o_to','o_asts','o_fgm','o_dreb'])
merged_df2 = merged_df2.drop(columns=['GP_player','GS','ftMade','threeAttempted','GP_team'])
merged_df2 = merged_df2.drop(columns=['o_3pm','o_3pa','o_reb','o_pf','o_stl','o_blk','d_fgm','d_fga','d_ftm','d_fta','d_3pm','d_3pa','d_oreb','d_dreb','d_reb','d_asts','d_pf','d_stl','d_to','d_blk','d_pts'])
merged_df2 = merged_df2.drop(columns=['PostGP','PostGS','PostMinutes','PostPoints','PostoRebounds','PostdRebounds','PostAssists','PostSteals','PostBlocks','PostTurnovers','PostPF','PostfgAttempted','PostfgMade','PostftAttempted','PostftMade','PostthreeAttempted','PostthreeMade','PostDQ'])
merged_df2 = merged_df2.drop(columns=['arena'])

merged_df2['birthDate'] = pd.to_datetime(merged_df['birthDate'], errors='coerce').dt.year
merged_df2 = merged_df2.rename(columns={'birthDate': 'birthYear'})

if not os.path.exists('cleanDatasets'):
    os.makedirs('cleanDatasets')

merged_df2.to_csv('cleanDatasets/advancedstatistics.csv', index=False)

In [245]:
players_stats_prevYear = merged_df2[['playerID','year','PER', 'eFG%', 'TS%','stocks','dRebounds']].drop_duplicates().copy()
players_stats_prevYear['year'] = players_stats_prevYear['year'] + 1


players_stats_prevYear = players_stats_prevYear.merge(
    merged_df2[['playerID', 'year', 'tmID', 'playoff']], 
    on=['playerID', 'year'], 
    how='left')

players_stats_prevYear.to_csv('cleanDatasets/players_stats_prevYear.csv', index=False)


In [246]:
# Calcula a média de PER, TS%, e eFG% por equipe e ano
team_year_stats = players_stats_prevYear.groupby(['tmID', 'year','playoff'])[['PER', 'TS%', 'eFG%']].mean().reset_index() # Tirei os stocks e o dRebounds

# Salva o novo dataset em um arquivo CSV
team_year_stats.to_csv('cleanDatasets/team_year_stats.csv', index=False)

In [247]:
#Dividir o dataset para treino, validacao e teste 

dataset_treino = team_year_stats[(team_year_stats['year'] >= 7) & (team_year_stats['year'] <= 9)]
dataset_teste = team_year_stats[team_year_stats['year'] == 10]

In [248]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

# ........................................... Para normalizar .....................................
# Selecionar as colunas para normalizar
cols_to_normalize = ['PER', 'TS%', 'eFG%']  # Tirei os stocks e o dRebounds

# Inicializar o MinMaxScaler
scaler = MinMaxScaler()

# Aplicar a normalização apenas nas colunas especificadas
team_year_stats[cols_to_normalize] = scaler.fit_transform(team_year_stats[cols_to_normalize])

# ...................................... Para fazer seleção de atributos com PCA ......................................
""" # Inicializar PCA e reduzir a dimensionalidade para 3 componentes principais
pca = PCA(n_components=3)

# Aplicar o PCA às colunas normalizadas
X_normalizado = team_year_stats[cols_to_normalize]  # Use as colunas normalizadas

# Transformar os dados
X_reduzido = pca.fit_transform(X_normalizado)

# Exibir a variância explicada pelas 3 primeiras componentes
print("Variância explicada pelas três primeiras componentes:", pca.explained_variance_ratio_)

# Se precisar verificar o conjunto reduzido de dados (as 3 primeiras componentes), você pode fazer:
print("Dados reduzidos com 3 componentes principais:")
print(X_reduzido)
 """

' # Inicializar PCA e reduzir a dimensionalidade para 3 componentes principais\npca = PCA(n_components=3)\n\n# Aplicar o PCA às colunas normalizadas\nX_normalizado = team_year_stats[cols_to_normalize]  # Use as colunas normalizadas\n\n# Transformar os dados\nX_reduzido = pca.fit_transform(X_normalizado)\n\n# Exibir a variância explicada pelas 3 primeiras componentes\nprint("Variância explicada pelas três primeiras componentes:", pca.explained_variance_ratio_)\n\n# Se precisar verificar o conjunto reduzido de dados (as 3 primeiras componentes), você pode fazer:\nprint("Dados reduzidos com 3 componentes principais:")\nprint(X_reduzido)\n '

In [249]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Features (X) e alvo (y)
X_treino = dataset_treino[['PER', 'TS%', 'eFG%']]
y_treino = dataset_treino['playoff']

X_teste = dataset_teste[['PER', 'TS%', 'eFG%']]
y_teste = dataset_teste['playoff']

In [250]:
# Inicializar o modelo de regressão
modelo = RandomForestRegressor(random_state=42)

# Treinar com os dados de treino
modelo.fit(X_treino, y_treino)

y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

print("Mean Squared Error (MSE) na validação:", mse)
print("R² score na validação:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) na validação: 0.2102923076923076
R² score na validação: 0.11151500000000047


/tmp/ipykernel_19824/3804687974.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.90
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.41
21,CON,10,0.0,18.172924,52.193001,47.665868,0.75
30,DET,10,1.0,17.456611,50.714573,44.487758,0.64
47,IND,10,1.0,14.525819,45.217894,40.307279,0.59
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.72
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.78
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.58
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.89
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.62


In [251]:
# Inicializar o modelo de regressão com Gradient Boosting
modelo = GradientBoostingRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.31103617016086105
R² score no teste: -0.3141278189296377


/tmp/ipykernel_19824/26995874.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.956449
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.878005
21,CON,10,0.0,18.172924,52.193001,47.665868,0.873907
30,DET,10,1.0,17.456611,50.714573,44.487758,0.680881
47,IND,10,1.0,14.525819,45.217894,40.307279,0.740968
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.902947
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.852928
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.810677
87,PHO,10,1.0,17.393405,49.871563,45.483028,1.017318
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.957079


In [252]:
# Inicializar o modelo de regressão linear
modelo = LinearRegression()

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.28516547608590526
R² score no teste: -0.2048241364629495


/tmp/ipykernel_19824/2857818035.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.834586
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.509512
21,CON,10,0.0,18.172924,52.193001,47.665868,0.817227
30,DET,10,1.0,17.456611,50.714573,44.487758,0.619942
47,IND,10,1.0,14.525819,45.217894,40.307279,0.486723
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.936019
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.979960
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.707439
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.775195
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.471349


In [253]:
#::::::::::::::::::::Este modelo é uma variante da regressão linear que usa regularização L2 para reduzir overfitting.::::::::::::::::

# Inicializar o modelo de Ridge Regression
modelo = Ridge(alpha=1.0)  # alpha controla o nível de regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.28507954861163104
R² score no teste: -0.20446109288414083


/tmp/ipykernel_19824/11041507.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.827071
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.508636
21,CON,10,0.0,18.172924,52.193001,47.665868,0.815056
30,DET,10,1.0,17.456611,50.714573,44.487758,0.627737
47,IND,10,1.0,14.525819,45.217894,40.307279,0.486976
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.929976
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.976384
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.705851
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.771346
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.481898


In [254]:
# Similar ao Ridge, mas utiliza regularização L1. Tende a eliminar variáveis menos importantes, útil para seleção de features.
# Inicializar o modelo de Lasso Regression
modelo = Lasso(alpha=0.1, random_state=42)  # alpha controla a força da regularização

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.26488175117231844
R² score no teste: -0.11912539870304517


/tmp/ipykernel_19824/613287734.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.683942
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.524758
21,CON,10,0.0,18.172924,52.193001,47.665868,0.719515
30,DET,10,1.0,17.456611,50.714573,44.487758,0.681362
47,IND,10,1.0,14.525819,45.217894,40.307279,0.525263
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.767686
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.818619
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.649819
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.677996
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.602798


In [255]:
# Combina as regularizações L1 (Lasso) e L2 (Ridge).
# Inicializar o modelo de Elastic Net
modelo = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
# `alpha` controla a força total da regularização.
# `l1_ratio` controla a proporção de regularização L1 (Lasso) em relação à L2 (Ridge).

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.2800517759907212
R² score no teste: -0.18321875356079653


/tmp/ipykernel_19824/260154350.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.717825
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.504407
21,CON,10,0.0,18.172924,52.193001,47.665868,0.766007
30,DET,10,1.0,17.456611,50.714573,44.487758,0.713856
47,IND,10,1.0,14.525819,45.217894,40.307279,0.503623
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.829718
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.898413
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.672739
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.709709
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.609372


In [256]:
# Inicializar o modelo de MLP
modelo = MLPRegressor(hidden_layer_sizes=(100, 50),  # Camadas ocultas com 100 e 50 neurônios
                      activation='relu',            # Função de ativação
                      solver='adam',                # Otimizador
                      max_iter=500,                 # Número máximo de iterações
                      random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 2.1154279850051814
R² score no teste: -7.93768323664689


/tmp/ipykernel_19824/2522216952.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,-0.799578
11,CHI,10,0.0,14.516353,48.523196,44.406606,-0.757605
21,CON,10,0.0,18.172924,52.193001,47.665868,-0.801149
30,DET,10,1.0,17.456611,50.714573,44.487758,-0.689464
47,IND,10,1.0,14.525819,45.217894,40.307279,-0.667695
56,LAS,10,1.0,19.077357,50.227263,45.677930,-0.773468
67,MIN,10,0.0,20.033617,52.258593,47.321910,-0.786267
76,NYL,10,0.0,16.864376,51.725000,47.477969,-0.804959
87,PHO,10,1.0,17.393405,49.871563,45.483028,-0.771713
98,SAC,10,0.0,15.981555,52.039114,46.109392,-0.720220


In [257]:
import numpy as np

# Inicializar o modelo sequencial
modelo = Sequential()

# Adicionar camadas
modelo.add(Dense(64, activation='relu', input_shape=(X_treino.shape[1],)))  # Primeira camada oculta
modelo.add(Dense(32, activation='relu'))  # Segunda camada oculta
modelo.add(Dense(1))  # Camada de saída (regressão)

# Compilar o modelo
modelo.compile(optimizer='adam', loss='mse')

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino, epochs=50, batch_size=32, verbose=1, validation_split=0.2)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste).flatten()  # Flatten para transformar em 1D

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

/home/emamartins12/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 997ms/step - loss: 198.0225 - val_loss: 170.4032
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 166.5372 - val_loss: 141.8376
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 138.4421 - val_loss: 116.2875
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 113.4044 - val_loss: 96.3948
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 94.0611 - val_loss: 79.9627
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 77.9303 - val_loss: 65.5465
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 63.7965 - val_loss: 53.0200
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 51.5113 - val_loss: 42.1100
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 40.8406 - val_loss: 32.7933
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 31.7677 - val_loss: 25.1463
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 24.2933 - val_loss: 19.2012
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - los

/tmp/ipykernel_19824/1782423946.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.586876
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.633962
21,CON,10,0.0,18.172924,52.193001,47.665868,0.639320
30,DET,10,1.0,17.456611,50.714573,44.487758,0.709988
47,IND,10,1.0,14.525819,45.217894,40.307279,0.627213
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.599638
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.627812
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.642291
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.615820
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.738279


In [258]:
# Inicializar o modelo de regressão com Extra Trees
modelo = ExtraTreesRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)

Mean Squared Error (MSE) no teste: 0.2185
R² score no teste: 0.07683750000000023


/tmp/ipykernel_19824/1732875705.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.94
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.41
21,CON,10,0.0,18.172924,52.193001,47.665868,0.83
30,DET,10,1.0,17.456611,50.714573,44.487758,0.64
47,IND,10,1.0,14.525819,45.217894,40.307279,0.28
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.78
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.74
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.52
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.87
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.50


In [259]:
# Inicializar o modelo ElasticNet
modelo = ElasticNet(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
print(dataset_teste.head(15))


Mean Squared Error (MSE) no teste: 0.23758751658811145
R² score no teste: -0.00380725758477074
    tmID  year  playoff        PER        TS%       eFG%  playoff_previsto
1    ATL    10      1.0  17.505053  49.771367  45.945109          0.585366
11   CHI    10      0.0  14.516353  48.523196  44.406606          0.585366
21   CON    10      0.0  18.172924  52.193001  47.665868          0.585366
30   DET    10      1.0  17.456611  50.714573  44.487758          0.585366
47   IND    10      1.0  14.525819  45.217894  40.307279          0.585366
56   LAS    10      1.0  19.077357  50.227263  45.677930          0.585366
67   MIN    10      0.0  20.033617  52.258593  47.321910          0.585366
76   NYL    10      0.0  16.864376  51.725000  47.477969          0.585366
87   PHO    10      1.0  17.393405  49.871563  45.483028          0.585366
98   SAC    10      0.0  15.981555  52.039114  46.109392          0.585366
105  SAS    10      1.0  16.491871  52.388268  47.734956          0.585366
114  

/tmp/ipykernel_19824/3962555249.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


In [260]:
# Inicializar o modelo XGBoost Regressor
modelo = xgb.XGBRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


Mean Squared Error (MSE) no teste: 0.30194847717048456
R² score no teste: -0.27573231604529713


/tmp/ipykernel_19824/1187298798.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.836160
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.721083
21,CON,10,0.0,18.172924,52.193001,47.665868,0.467712
30,DET,10,1.0,17.456611,50.714573,44.487758,0.509861
47,IND,10,1.0,14.525819,45.217894,40.307279,0.960736
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.726918
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.777833
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.739202
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.850995
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.578495


In [261]:
# Inicializar o modelo LightGBM Regressor
modelo = lgb.LGBMRegressor(random_state=42)

# Treinar o modelo com os dados de treino
modelo.fit(X_treino, y_treino)

# Fazer previsões no conjunto de teste
y_pred_teste = modelo.predict(X_teste)

# Avaliar o modelo com métricas de regressão
mse = mean_squared_error(y_teste, y_pred_teste)
r2 = r2_score(y_teste, y_pred_teste)

# Exibir as métricas
print("Mean Squared Error (MSE) no teste:", mse)
print("R² score no teste:", r2)

# Adicionar a coluna prevista ao dataset_teste
dataset_teste['playoff_previsto'] = y_pred_teste

# Exibir as primeiras linhas para verificar
dataset_teste.head(15)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027813 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 45
[LightGBM] [Info] Number of data points in the train set: 41, number of used features: 3
[LightGBM] [Info] Start training from score 0.585366
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

/tmp/ipykernel_19824/3421237157.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_teste['playoff_previsto'] = y_pred_teste


,tmID,year,playoff,PER,TS%,eFG%,playoff_previsto
1,ATL,10,1.0,17.505053,49.771367,45.945109,0.628851
11,CHI,10,0.0,14.516353,48.523196,44.406606,0.419221
21,CON,10,0.0,18.172924,52.193001,47.665868,0.628851
30,DET,10,1.0,17.456611,50.714573,44.487758,0.628851
47,IND,10,1.0,14.525819,45.217894,40.307279,0.543951
56,LAS,10,1.0,19.077357,50.227263,45.677930,0.628851
67,MIN,10,0.0,20.033617,52.258593,47.321910,0.628851
76,NYL,10,0.0,16.864376,51.725000,47.477969,0.628851
87,PHO,10,1.0,17.393405,49.871563,45.483028,0.628851
98,SAC,10,0.0,15.981555,52.039114,46.109392,0.628851
